*READING DATA FROM WANDB*

In [2]:
import wandb
import pandas as pd

run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)

wandb: Currently logged in as: fonsogv5. Use `wandb login --relogin` to force relogin


*INITIAL PROFILE*

In [4]:
import pandas_profiling

profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

AttributeError: module 'seaborn' has no attribute 'set_style'

*SOLVING SOME PROBLEMS*

In [6]:
# Drop outliers
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()
# Convert last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

*ENDING EDA*

In [7]:
run.finish()